# Build the networks from the data Retrieved by the databases 

In [1]:
import networkx as nx
from fa2 import ForceAtlas2  #the package should be installed before
import matplotlib.pyplot as plt
import datetime
from statistics import mean, median, quantiles
from networkx.algorithms.traversal.breadth_first_search import descendants_at_distance
import time
import requests
import glob
import pickle
import json
import zstandard
import pandas as pd
import os
import shutil

In [2]:
def replies_of(row, replies, s_k):
    if replies.shape[0] == 0:
        return
    for index_c, row_c in replies.iterrows():
        commentsGraph.add_node(row_c["id"], subset_key = s_k, created=row_c["created_utc"], body=row_c["body"],author=row_c["author_fullname"],score=row_c["score"])
        commentsGraph.add_edge(row_c["parent_id"], row_c["id"])
            
        authorGraph.add_node(row_c["author_fullname"], created=row_c["created_utc"])
        authorGraph.add_edge(row_c["author_fullname"], row["author_fullname"])
        
        replies_of(row_c, comm_df[comm_df["parent_id"]==row_c["id"]], s_k+1)

In [6]:
subreddit = "PoliticalDiscussion"

shutil.rmtree("aG_prove_monthly_"+subreddit+"/aux")
os.mkdir("aG_prove_monthly_"+subreddit+"/aux")
shutil.rmtree("cG_prove_monthly_"+subreddit+"/aux")
os.mkdir("cG_prove_monthly_"+subreddit+"/aux")

shutil.rmtree("aG_prove_monthly_"+subreddit+"/total")
os.mkdir("aG_prove_monthly_"+subreddit+"/total")
shutil.rmtree("cG_prove_monthly_"+subreddit+"/total")
os.mkdir("cG_prove_monthly_"+subreddit+"/total")



In [7]:
subreddit = "PoliticalDiscussion"
subfolders = ["2019-2","2019-23"]

for subfolder in subfolders:
    shutil.rmtree("aG_prove_monthly_"+subreddit+"/"+subfolder)
    os.mkdir("aG_prove_monthly_"+subreddit+"/"+subfolder)
    shutil.rmtree("cG_prove_monthly_"+subreddit+"/"+subfolder)
    os.mkdir("cG_prove_monthly_"+subreddit+"/"+subfolder)
    
    program_starts = time.time()
    in_df = pd.read_pickle("RS_"+subreddit+"_"+subfolder+".pickle")
    print()
    print(subfolder)
    print("total: ",in_df.shape[0])
    
    
    for index, row in in_df.iterrows():
        authorGraph = nx.DiGraph()
        commentsGraph = nx.DiGraph()
        commentsGraph.add_node(row["name"], subset_key=0, num_comm=row["num_comments"], created=row["created_utc"], body=row["title"],author=row["author_fullname"],score=row["score"])
        authorGraph.add_node(row["author_fullname"], subset_key=0, created=row["created_utc"])
    
        nx.write_gpickle(authorGraph,"aG_prove_monthly_"+subreddit+"/aux/aGraph_"+row["name"]+"_"+subreddit)
        nx.write_gpickle(commentsGraph,"cG_prove_monthly_"+subreddit+"/aux/cGraph_"+row["name"]+"_"+subreddit)
        
    comm_df = pd.read_pickle("RC_"+subreddit+"_"+subfolder+".pickle")
    
    
    
    '''
    !!!!!!!!!!!!!!!!!!!!!
    '''

    moderator = 1
    
    '''
    !!!!!!!!!!!!!!!!!!!!!
    '''




    for index, row in in_df.iterrows():
        if index%500 == 0 and index!=0:
            now = time.time()
            print("Done: ",index,
                  "\tin ",str(datetime.timedelta(seconds=(now - program_starts))).split('.')[0])
        aux = comm_df[comm_df["parent_id"]==row["name"]]
        if aux.shape[0] > moderator:
            commentsGraph = nx.read_gpickle("cG_prove_monthly_"+subreddit+"/aux/cGraph_"+row["name"]+"_"+subreddit)
            authorGraph = nx.read_gpickle("aG_prove_monthly_"+subreddit+"/aux/aGraph_"+row["name"]+"_"+subreddit)
        
            for index_c, row_c in aux.iterrows():
                commentsGraph.add_node(row_c["id"], subset_key=1, created=row_c["created_utc"], body=row_c["body"],author=row_c["author_fullname"],score=row_c["score"])
                commentsGraph.add_edge(row_c["parent_id"], row_c["id"])
                
                if row_c["author_fullname"] not in authorGraph:
                    authorGraph.add_node(row_c["author_fullname"], created=row_c["created_utc"])
                authorGraph.add_edge(row_c["author_fullname"], row["author_fullname"])
            
                replies_of(row_c, comm_df[comm_df["parent_id"]==row_c["id"]], s_k=1 )
            
            nx.write_gpickle(authorGraph,"aG_prove_monthly_"+subreddit+"/"+subfolder+"/aGraph_"+row["name"]+"_"+subreddit)
            nx.write_gpickle(commentsGraph,"cG_prove_monthly_"+subreddit+"/"+subfolder+"/cGraph_"+row["name"]+"_"+subreddit)
   


2019-2
total:  6664
Done:  500 	in  0:01:20
Done:  1000 	in  0:02:12
Done:  1500 	in  0:03:27
Done:  2000 	in  0:04:06
Done:  2500 	in  0:05:00
Done:  3000 	in  0:06:51
Done:  3500 	in  0:08:09
Done:  4000 	in  0:09:11
Done:  4500 	in  0:10:26
Done:  5000 	in  0:11:01
Done:  5500 	in  0:11:28
Done:  6000 	in  0:12:01
Done:  6500 	in  0:12:49

2019-23
total:  8266
Done:  500 	in  0:02:21
Done:  1000 	in  0:06:59
Done:  1500 	in  0:10:08
Done:  2000 	in  0:12:58
Done:  2500 	in  0:18:39
Done:  3000 	in  0:22:11
Done:  3500 	in  0:25:21
Done:  4000 	in  0:28:32
Done:  4500 	in  0:33:17
Done:  5000 	in  0:36:17
Done:  5500 	in  0:38:13
Done:  6000 	in  0:39:37
Done:  6500 	in  0:41:32
Done:  7000 	in  0:42:42
Done:  7500 	in  0:43:50
Done:  8000 	in  0:45:14


In [8]:
subreddit = "PoliticalDiscussion"
subfolders = ["2019-2","2019-23","total_aux"]

for subfolder in subfolders:
    pathsA = glob.glob("aG_prove_monthly_"+subreddit+"/"+subfolder+"/*") 
    pathsC = glob.glob("cG_prove_monthly_"+subreddit+"/"+subfolder+"/*") 
    for f in pathsA:
        shutil.copy(f, "aG_prove_monthly_"+subreddit+"/total")
    for f in pathsC:
        shutil.copy(f, "cG_prove_monthly_"+subreddit+"/total")